In [ ]:
def read_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    data = [list(map(float, line.strip().split('\t'))) for line in lines]
    return data

def compose_matrix(data):
    n = len(data)
    X = [[1.0] for _ in range(n)]
    
    for i in range(n):
        for j in range(5):
            X[i].append(data[i][j])
            
    for i in range(n):
        for j in range(i, 5):
            for k in range(n):
                X[k].append(data[k][i] * data[k][j])
    
    return X

def transpose(matrix):
    return [list(row) for row in zip(*matrix)]

def matrix_multiply(A, B):
    result = [[0] * len(B[0]) for _ in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            result[i][j] = sum(A[i][k] * B[k][j] for k in range(len(B)))
    return result

def gauss_jordan(A, b):
    n = len(A)
    M = [A[i] + [b[i]] for i in range(n)]
    
    for i in range(n):
        max_row = max(range(i, n), key=lambda r: abs(M[r][i]))
        M[i], M[max_row] = M[max_row], M[i]
        
        if M[i][i] == 0:
            for k in range(i + 1, n):
                if M[k][i] != 0:
                    M[i], M[k] = M[k], M[i]
                    break

        div = M[i][i]
        if div != 0:
            M[i] = [x / div for x in M[i]]
        
        for j in range(n):
            if i != j:
                factor = M[j][i]
                M[j] = [M[j][k] - factor * M[i][k] for k in range(len(M[0]))]
    
    return [row[-1] for row in M]



def linear_regression(X, y):
    XT = transpose(X)
    XTX = matrix_multiply(XT, X)
    XTy = matrix_multiply(XT, [[yi] for yi in y])
    XTy = [row[0] for row in XTy]
    return gauss_jordan(XTX, XTy)

def predict(X, w):
    return [sum(X[i][j] * w[j] for j in range(len(w))) for i in range(len(X))]


data = read_data('input.txt')

train_points = [row[:5] for row in data[:1000]]
train_values = [row[5] for row in data[:1000]]
test_points = [row[:5] for row in data[1000:]]

X_train = compose_matrix(train_points)

coefficients = linear_regression(X_train, train_values)

X_test = compose_matrix(test_points)

predictions = predict(X_test, coefficients)

for prediction in predictions:
    print(prediction)